In [1]:
import pandas as pd
import numpy as np
from datetime import datetime


In [2]:
crime_csv_path = "/Users/mignonduplessis/Downloads/Chicago_Crimes_2012_to_2017.csv"
weather_csv_path = "/Users/mignonduplessis/Documents/BootCampRep/row3/ChicagoDailyWeather.csv"

In [3]:
weather_df = pd.read_csv(weather_csv_path)
crimes_df = pd.read_csv(crime_csv_path)

In [4]:
weather_df['DATE']=pd.to_datetime(weather_df['DATE'], format='%Y/%m/%d')

date1 = datetime.strptime('2012-01-01', '%Y-%m-%d')
date2 = datetime.strptime('2016-12-31', '%Y-%m-%d')

weather = weather_df.loc[(weather_df['DATE']>=date1) & (weather_df['DATE']<=date2)]
org_weather = weather.drop(['AWND', 'WDF2','STATION','FMTM', 'NAME','TAVG','TSUN', 'WDF5', 'WSF2', 'WSF5','WT01','WT02',
                            'WT03','WT04','WT05','WT08','WT09'], axis = 1)
new_weather = org_weather.rename(columns={"DATE": "Date"})
new_weather.head()

,Date,PGTM,PRCP,TMAX,TMIN
4017,2012-01-01,1103.0,0.02,42.0,28.0
4018,2012-01-02,1014.0,0.00,28.0,16.0
4019,2012-01-03,2042.0,0.00,27.0,10.0
4020,2012-01-04,1326.0,0.00,41.0,22.0
4021,2012-01-05,2359.0,0.00,49.0,21.0


In [5]:
new_weather.dtypes

Date    datetime64[ns]
PGTM           float64
PRCP           float64
TMAX           float64
TMIN           float64
dtype: object

In [6]:
#org_weather = weather_df[['DATE', 'TAVG', 'TMAX', 'TMIN']].set_index(['DATE'])
#weather = org_weather.loc['2012-01-01':'2017-12-31']
#weather.head()

In [7]:
crimes_df.dtypes

Unnamed: 0                int64
ID                        int64
Case Number              object
Date                     object
Block                    object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Beat                      int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                 object
X Coordinate            float64
Y Coordinate            float64
Year                      int64
Updated On               object
Latitude                float64
Longitude               float64
Location                 object
dtype: object

In [8]:
crimes_df.columns
org_crimes = crimes_df.drop(['Ward', 'Community Area', 'FBI Code',
       'X Coordinate', 'Y Coordinate', 'Year', 'Updated On', 'Latitude',
       'Longitude', 'Location', 'Unnamed: 0', 'ID', 'Block', 'IUCR', 'Beat', 'District'], axis = 1)


org_crimes.head()


,Case Number,Date,Primary Type,Description,Location Description,Arrest,Domestic
0,HZ250496,05/03/2016 11:40:00 PM,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,True
1,HZ250409,05/03/2016 09:40:00 PM,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True
2,HZ250503,05/03/2016 11:31:00 PM,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,STREET,False,False
3,HZ250424,05/03/2016 10:10:00 PM,BATTERY,SIMPLE,SIDEWALK,False,False
4,HZ250455,05/03/2016 10:00:00 PM,THEFT,$500 AND UNDER,RESIDENCE,False,True


In [9]:
#org_crimes.dtypes

In [10]:
org_crimes['Date'] = pd.to_datetime(org_crimes['Date'])
org_crimes.head()

,Case Number,Date,Primary Type,Description,Location Description,Arrest,Domestic
0,HZ250496,2016-05-03 23:40:00,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,True
1,HZ250409,2016-05-03 21:40:00,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True
2,HZ250503,2016-05-03 23:31:00,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,STREET,False,False
3,HZ250424,2016-05-03 22:10:00,BATTERY,SIMPLE,SIDEWALK,False,False
4,HZ250455,2016-05-03 22:00:00,THEFT,$500 AND UNDER,RESIDENCE,False,True


In [11]:
org_crimes.dtypes

Case Number                     object
Date                    datetime64[ns]
Primary Type                    object
Description                     object
Location Description            object
Arrest                            bool
Domestic                          bool
dtype: object

In [12]:
org_crimes['Date'] = pd.to_datetime(org_crimes['Date']).dt.date
org_crimes.dtypes

Case Number             object
Date                    object
Primary Type            object
Description             object
Location Description    object
Arrest                    bool
Domestic                  bool
dtype: object

In [13]:
org_crimes['Date'] = pd.to_datetime(org_crimes['Date'])
org_crimes.dtypes

Case Number                     object
Date                    datetime64[ns]
Primary Type                    object
Description                     object
Location Description            object
Arrest                            bool
Domestic                          bool
dtype: object

In [14]:
## Merging 2 Datasets

weather_crime = pd.merge(org_crimes,new_weather, on="Date", how = 'outer')
weather_crime.dropna(how = 'any')
weather_crime.count()

Case Number             1456713
Date                    1456714
Primary Type            1456714
Description             1456714
Location Description    1455056
Arrest                  1456714
Domestic                1456714
PGTM                    1444464
PRCP                    1441385
TMAX                    1440929
TMIN                    1440929
dtype: int64

In [15]:
weather_crime = weather_crime.dropna(how = 'any')
weather_crime.count()

Case Number             1436221
Date                    1436221
Primary Type            1436221
Description             1436221
Location Description    1436221
Arrest                  1436221
Domestic                1436221
PGTM                    1436221
PRCP                    1436221
TMAX                    1436221
TMIN                    1436221
dtype: int64

In [16]:
weather_crime.head()

,Case Number,Date,Primary Type,Description,Location Description,Arrest,Domestic,PGTM,PRCP,TMAX,TMIN
0,HZ250496,2016-05-03,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,True,1531.0,0.0,67.0,36.0
1,HZ250409,2016-05-03,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,1531.0,0.0,67.0,36.0
2,HZ250503,2016-05-03,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,STREET,False,False,1531.0,0.0,67.0,36.0
3,HZ250424,2016-05-03,BATTERY,SIMPLE,SIDEWALK,False,False,1531.0,0.0,67.0,36.0
4,HZ250455,2016-05-03,THEFT,$500 AND UNDER,RESIDENCE,False,True,1531.0,0.0,67.0,36.0


In [26]:
#Get monthly high temperatures over the course of the 5 years
avg_df = weather_crime.loc[:,["Date", "TMAX"]]
avg_df['Year'] = pd.DatetimeIndex(avg_df["Date"]).year
avg_df['Month'] = pd.DatetimeIndex(avg_df["Date"]).month

Monthly_high = pd.DataFrame(avg_df.groupby(['Year', 'Month']).mean()['TMAX'])
Monthly_high.rename(columns = {'TMAX':'Monthly avg TMAX'}, inplace = True)

Monthly_high.head()
 

Monthly avg TMAX
Year Month                  
2012 1             37.872765
     2             39.165141
     3             63.459923
     4             59.896699
     5             75.887140

In [70]:
#get the daily avg temp
avg_daily = weather_crime.loc[:,["Date", "TMAX", "TMIN"]]
maxtemp = weather_crime["TMAX"]
mintemp = weather_crime["TMIN"]
total_avg = maxtemp + mintemp / 2
avg_daily['Daily Average Temp'] = total_avg 
avg_daily.head()

,Date,TMAX,TMIN,Daily Average Temp
0,2016-05-03,67.0,36.0,85.0
1,2016-05-03,67.0,36.0,85.0
2,2016-05-03,67.0,36.0,85.0
3,2016-05-03,67.0,36.0,85.0
4,2016-05-03,67.0,36.0,85.0


In [68]:
#5 Categories (Cold, Mild, Warm, Very Warm, Hot) 
#Below 32 Degrees Fahrenheit : Cold
#33 to 65 Degrees Fahrenheit: Mild
#66 to 75 Degrees Fahrenheit: Warm
#76 to 85 Degrees Fahrenheit: Very Warm
#Above 85 Degrees: Hot

#create function to filter through data

def f(row):

    if row['Daily Average Temp'] <= 32:

        val = 'Cold'

    elif row['Daily Average Temp'] <= 65:

        val = 'Mild'
        
    elif row['Daily Average Temp'] <= 75:

        val = 'Warm'
        
    elif row['Daily Average Temp'] <= 85:

        val = 'Very warm'

    else:

        val = 'Hot'

    return val



In [72]:
#apply function & create new column for the return values
avg_daily['Category'] = avg_daily.apply(f, axis=1)
avg_daily.head()


,Date,TMAX,TMIN,Daily Average Temp,Category
0,2016-05-03,67.0,36.0,85.0,Very warm
1,2016-05-03,67.0,36.0,85.0,Very warm
2,2016-05-03,67.0,36.0,85.0,Very warm
3,2016-05-03,67.0,36.0,85.0,Very warm
4,2016-05-03,67.0,36.0,85.0,Very warm


In [73]:
#4 Categories (Fall, Winter, Spring, Summer) 
#Fall: 09 to 11 
#Winter: 11 to 3
#Spring: 3 to 5
#Summer: 06 to 8


#adding year and month to average daily weather dataframe to create the season by month dataframe
season_df = avg_daily.loc[:,["Date", "TMAX","TMIN","Daily Average Temp","Category"]]
season_df['Year'] = pd.DatetimeIndex(season_df["Date"]).year
season_df['Month'] = pd.DatetimeIndex(season_df["Date"]).month

season_df.head()

,Date,TMAX,TMIN,Daily Average Temp,Category,Year,Month
0,2016-05-03,67.0,36.0,85.0,Very warm,2016,5
1,2016-05-03,67.0,36.0,85.0,Very warm,2016,5
2,2016-05-03,67.0,36.0,85.0,Very warm,2016,5
3,2016-05-03,67.0,36.0,85.0,Very warm,2016,5
4,2016-05-03,67.0,36.0,85.0,Very warm,2016,5


In [74]:
#create function to filter through data

def I(row):

    if row['Month'] <=3:

        val = 'Winter'

    elif row['Month'] <= 5:

        val = 'Spring'
        
    elif row['Month'] <= 8:

        val = 'Summer'
        
    elif row['Month'] <= 11:

        val = 'Fall'

    else:

        val = 'Winter'

    return val


In [76]:
#apply function & create new column for the return values

season_df['Season'] = season_df.apply(I, axis=1)
season_df.head()

,Date,TMAX,TMIN,Daily Average Temp,Category,Year,Month,Season
0,2016-05-03,67.0,36.0,85.0,Very warm,2016,5,Spring
1,2016-05-03,67.0,36.0,85.0,Very warm,2016,5,Spring
2,2016-05-03,67.0,36.0,85.0,Very warm,2016,5,Spring
3,2016-05-03,67.0,36.0,85.0,Very warm,2016,5,Spring
4,2016-05-03,67.0,36.0,85.0,Very warm,2016,5,Spring
